In [ ]:
#라이브러리 세팅, 파일 다운로드 후 압축풀기
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


path_to_zip = tf.keras.utils.get_file(
    '/content/spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)


In [ ]:
!unzip /content/spa-eng.zip

Archive:  /content/spa-eng.zip
replace spa-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: spa-eng/_about.txt      
  inflating: spa-eng/spa.txt         


In [ ]:
!head /content/spa-eng/spa.txt

Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.
Run!	¡Corre!
Run.	Corred.
Who?	¿Quién?
Fire!	¡Fuego!
Fire!	¡Incendio!


In [ ]:
#판다스로 읽어들이기

lines = pd.read_csv('/content/spa-eng/spa.txt', names=['src', 'tar'], sep='\t')


In [ ]:
len(lines)

118964

In [ ]:
lines = lines[0:30000] # 3만개만 저장

In [ ]:
lines.head()

,src,tar
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.


In [ ]:
lines.tar = lines.tar.apply(lambda x : '\t' + x + '\n')
lines.head()

# start <SOS> = \t
# END <EOS> = \n
# <UNK>


,src,tar
0,Go.,\tVe.\n
1,Go.,\tVete.\n
2,Go.,\tVaya.\n
3,Go.,\tVáyase.\n
4,Hi.,\tHola.\n


In [ ]:
src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)



In [ ]:
len(src_vocab) ,len(tar_vocab)

(76, 93)

In [ ]:
src_vocab = sorted(src_vocab)
tar_vocab = sorted(tar_vocab)

In [ ]:
src_to_index = {}
tar_to_index = {}
for i , v in enumerate(src_vocab):
    src_to_index[v] = i + 1

for i , v in enumerate(tar_vocab):
    tar_to_index[v] = i + 1


In [ ]:
tar_to_index

{'\t': 1,
 '\n': 2,
 ' ': 3,
 '!': 4,
 '"': 5,
 '$': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '/': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 'a': 52,
 'b': 53,
 'c': 54,
 'd': 55,
 'e': 56,
 'f': 57,
 'g': 58,
 'h': 59,
 'i': 60,
 'j': 61,
 'k': 62,
 'l': 63,
 'm': 64,
 'n': 65,
 'o': 66,
 'p': 67,
 'q': 68,
 'r': 69,
 's': 70,
 't': 71,
 'u': 72,
 'v': 73,
 'w': 74,
 'x': 75,
 'y': 76,
 'z': 77,
 '¡': 78,
 '«': 79,
 '»': 80,
 '¿': 81,
 'Á': 82,
 'É': 83,
 'Ó': 84,
 'Ú': 85,
 'á': 86,
 'é': 87,
 'í': 88,
 'ñ': 89,
 'ó': 90,
 'ú': 91,
 'ü': 92,
 '€': 93}

In [ ]:
# encoder : 인풋으로 들어오는 타겟 언어 받는곳
# decoder : 아웃풋으로 나오는 타겟 언어 출력하는곳
encoder_input = []

for line in lines.src:
    encoder_line = []

    for char in line:
        encoder_line.append(src_to_index[char])
        
    encoder_input.append(encoder_line)


In [ ]:
decoder_input = []

for line in lines.tar:
    decoder_line = []

    for char in line:
        decoder_line.append(tar_to_index[char])
        
    decoder_input.append(decoder_line)


In [ ]:
# LSTM 한줄로 쭉 연결해서 하는 경우
# 교사 강요 - 직접적으로 학습하며 정답을 알려주는 경우
# 교사 강요 - 디코더의 아웃풋이 다음 디코더의 인풋으로 받는 경우

In [ ]:
decoder_input[:5] , encoder_input[:5]

([[1, 47, 56, 12, 2],
  [1, 47, 56, 71, 56, 12, 2],
  [1, 47, 52, 76, 52, 12, 2],
  [1, 47, 86, 76, 52, 70, 56, 12, 2],
  [1, 33, 66, 63, 52, 12, 2]],
 [[28, 62, 8], [28, 62, 8], [28, 62, 8], [28, 62, 8], [29, 56, 8]])

In [ ]:
decoder_output = []

for line in lines.tar:
    decoder_line = []
    for char in line[1:]:
        decoder_line.append(tar_to_index[char])
        
    decoder_output.append(decoder_line)


In [ ]:
decoder_output[:5]

[[47, 56, 12, 2],
 [47, 56, 71, 56, 12, 2],
 [47, 52, 76, 52, 12, 2],
 [47, 86, 76, 52, 70, 56, 12, 2],
 [33, 66, 63, 52, 12, 2]]

In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 70


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len,padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_output = pad_sequences(decoder_output, maxlen=max_tar_len, padding='post')

In [ ]:
encoder_input[0]

array([28, 62,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0], dtype=int32)

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_output = to_categorical(decoder_output)

In [ ]:
decoder_output[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
src_vocab_size = len(src_vocab) +1
tar_vocab_size = len(tar_vocab) +1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 77
target 문장의 char 집합 : 94


In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy")

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_output, batch_size=64, epochs=40, validation_split=0.2)


In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 21ms/step
-----------------------------------
입력 문장: Go.
정답 문장: áyase.
번역 문장: e.
1/1 [==============================] - 0s 27ms/step
-----------------------------------
입력 문장: I'm up.
정답 문장: stoy levantado.
번역 문장: stoy correr.
1/1 [==============================] - 0s 20ms/step
-----------------------------------
입력 문장: Go away!
정답 문장: Salga de aquí!
번역 문장: Vete de aquí!
1/1 [==============================] - 0s 22ms/step
-----------------------------------
입력 문장: I'm back.
정답 문장: e vuelto.
번역 문장: stoy de la cama.
1/1 [==============================] - 0s 32ms/step
-----------------------------------
입력 문장: He is here!
정답 문장: Él está aquí!
번역 문장: Él está aquí!


In [ ]:
# 나 / 는/ 배 /가
# 나  = [ 1,2,3,4,5] = 
# 는  = [ 1,2,3,4,5]
# 배  = [ 1,2,3,4,5]
# 가  = [ 1,2,3,4,5]